# Manipulating Data

Son Huynh
31.01.2020

## Pandas Documentation:
- Working with date: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html
- Datetime Index: https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DatetimeIndex.html

## Table of content:
* [1. Data Cleaning](#1.-Data-Cleaning)
* [2. Conditional transformation](#2.-Conditional-Transformation)
* [3. Time Series transformation](#3.-Time-series-Transformation)
* [4. Group transformation](#4.-Group-Transformation)

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Data Cleaning

Guideline on how to approach data cleaning and validating: https://www.kaggle.com/sohier/structured-eda-for-data-cleaning

In [ ]:
aapl = pd.read_csv('AAPL.csv')

aapl.head()

In [ ]:
# The dataset contains some missing values
aapl.info()

#### Convert Date column

In [ ]:
# Recommended: use pd.to_datetime() and declare the input format explicitly
aapl.date = pd.to_datetime(aapl.date, format='%Y/%m/%d', errors='coerce')

The format convention can be seen here:

https://docs.python.org/2/library/datetime.html#strftime-strptime-behavior

You can use `aapl.date.astype('datetime64')` as well, but `pd.to_datetime()` is faster, more robust and you can specify the format and error handling.

#### Convert string to number

In [ ]:
aapl.close = aapl.close.str.replace('$', '')

In [ ]:
aapl.close = pd.to_numeric(aapl.close, errors='coerce')

You can use `.astype(float)` here as well. However, the advantage of `pd.to_numeric()` is that you can specify `errors='coerce'`, which will convert all non-number / missing values (NaN) to `NaN` for you.

#### Checking missing data

In [ ]:
aapl[aapl.date.isnull()]

In [ ]:
aapl[aapl.isnull().any(axis=1)] # select rows with missing data in any column

In [ ]:
aapl[aapl.isnull().all(axis=1)] # only rows with missing data in all columns

#### Remove rows with missing data in one or many columns

In [ ]:
aapl = aapl.dropna(subset=['date'])

In [ ]:
aapl = aapl.dropna(thresh=3) # only keep rows with data in at least 3 columns

In [ ]:
aapl.info()

#### Fill missing data with some value

There is no best way to fill or drop missing data. Sometimes missing data contain information, thus it is not okay to just drop them. There are multiple approaches to fill and impute missing data. Usually an analyst have to apply logics and knowledge of the data to choose the appropriate method.

https://towardsdatascience.com/missing-values-dont-drop-them-f01b1d8ff557

https://medium.com/ibm-data-science-experience/missing-data-conundrum-exploration-and-imputation-techniques-9f40abe0fd87

In [ ]:
# aapl = aapl.fillna(0) # Fill missing values with zero.

In [ ]:
# Use last available value to fill. Sort_index before this step is necessary.
aapl = aapl.set_index('date').sort_index()
aapl = aapl.fillna(method='ffill')

# You can fill the entire dataframe, or choose one column to apply fillna().

In [ ]:
# The high and low columns on April 21 have been filled with data from April 20

aapl.loc['2015-04-20':'2015-04-22']

#### Detecting outliers by descriptive statistic and plotting

In [ ]:
aapl.describe() # Some anomaly in max close price

In [ ]:
aapl.close.plot(kind='line') # Some price is too high, must be errors

In [ ]:
aapl[(aapl.close>300)] # These are the rows to be corrected. Looks like they were multiplied by 100 by mistake.

## 2. Conditional Transformation

#### np.where() is like for-loop + if-else but vectorized.

In [ ]:
# Let's correct the outliers previously spotted.
aapl.close = np.where((aapl.close > 300), aapl.close/100, aapl.close)

In [ ]:
aapl.close.plot(kind='line') # Now it looks more correct

#### Using np.where() to create new column

In [ ]:
aapl['direction'] = np.where((aapl.close > aapl.open), 'up',
                                np.where((aapl.close < aapl.open), 'down', 'unchanged'))

aapl.head()

#### Binning with pd.cut()

Turn continuous data into categories. Useful for things like converting age to age groups.

Check out also pd.qcut()

In [ ]:
bins = [0, 25000000, 50000000, 100000000, 200000000]
labels = ['low', 'medium', 'high', 'very high']

In [ ]:
aapl['volume_cat'] = pd.cut(x=aapl.volume, bins=bins, labels=labels)

aapl

## 3. Time series Transformation

#### diff()
Also check out `shift()` and `pct_change()`

In [ ]:
aapl['profit'] = aapl.close.diff(periods=1)

aapl.tail()

#### Day of week

Datetime index has some special methods

In [ ]:
aapl['day_of_week'] = aapl.index.day_name()

aapl.head()

#### Resample
Resample is just like groupby, but for datetime index.
You can use resample if you have a datetime index.

In [ ]:
aapl.resample('M').first()

In [ ]:
aapl.resample('Y').mean()

## 4. Structure Transformation

#### Stack() and unstack()
Stack/unstack is very useful when you want to transform the structure of your dataframe. Check out `transpose()` also

In [ ]:
stacked = aapl.stack()
stacked # returns a series with 2-levels index

In [ ]:
unstacked = stacked.unstack(level=0) # you can specify which level in the index you want to turn into columns.
unstacked

In [ ]:
aapl.transpose().stack() # Try playing with transpose and stack/unstack to understand them

#### Pivot_table()
Pivot is great for quick view report on grouped data. It is like a 2 dimension groupby operation. The default groupby function is mean.

In [ ]:
aapl.pivot_table(index='day_of_week', columns='volume_cat', values='close', aggfunc='mean')